In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None) # set the max columns to display to none
#Private data source: https://www.sbe.wa.gov/our-work/private-schools
d2018 = pd.read_excel('WA_privateschools/Final Excel P105B Combined Data for 201819.xlsx')
d2019 = pd.read_excel('WA_privateschools/Copy of 2019-20 Private School Enrollment_073120.xlsx', sheet_name=1, skiprows=[0,1])
#d2020 = pd.read_excel('WA_privateschools/Copy of 2020-21 Private School Enrollment x Grade_for the website.xlsx')
d2020 = pd.read_excel('WA_privateschools/Fixed - Copy of 2020-21 Private School Enrollment x Grade_for the website.xlsx')
d2021 = pd.read_excel('WA_privateschools/2021-22 Private School Enrollment (Website).xlsx')
d2022 = pd.read_excel('WA_privateschools/2022 - 2023 Private School Enrollment (website).xlsx')
d2023 = pd.read_excel('WA_privateschools/2023 - 2024 Private School Enrollment (website).xlsx')

regionSubstring = "ellevue"  # Bellevue et al.  Replace with other districts and re-run if you wish

In [ ]:
gradeMap = {'Total PreK':'P',
 'Total KG':'K',
 'Total G1':'1',
 'Total G2':'2',
 'Total G3':'3',
 'Total G4':'4',
 'Total G5':'5',
 'Total G6':'6',
 'Total G7':'7',
 'Total G8':'8',
 'Total G9':'9',
 'Total G10':'10',
 'Total G11':'11',
 'Total G12':'12',
 'PK Total': 'P',
#        'PK Total.1',
 'KG Total': 'K',
 'G1 Total': '1',
 'G2 Total': '2',
 'G3 Total': '3',
 'G4 Total': '4',
 'G5 Total': '5',
 'G6 Total': '6',
 'G7 Total': '7',
 'G8 Total': '8',
 'G9 Total': '9',
 'G10 Total': '10',
 'G11 Total': '11',
 'G12 Total': '12',

 'PreK TOTAL': 'P',
 'KG TOTAL': 'K',
 'G1 TOTAL': '1',
 'G2 TOTAL': '2',
 'G3 TOTAL': '3',
 'G4 TOTAL': '4',
 'G5 TOTAL': '5',
 'G6 TOTAL': '6',
 'G7 TOTAL': '7',
 'G8 TOTAL': '8',
 'G9 TOTAL': '9',
 'G10 TOTAL': '10',
 'GI0 TOTAL': '10',
 'G11 TOTAL': '11',
 'G12 TOTAL': '12',

 'PreK': 'P',
 'KG': 'K',
 'Grade 1':'1',
 'Grade2':'2',
 'Grade 2':'2',
 'Grade 3':'3',
 'Grade 4':'4',
 'Grade 5':'5',
 'Grade 6':'6',
 'Grade 7':'7',
 'Grade 8':'8',
 'Grade 9':'9',
 'Grade 10':'10',
 'Grade 11':'11',
 'Grade 12':'12',            
  1:'1', 2:'2', 3:'3', 4:'4', 5:'5', 6:'6', 7:'7', 8:'8', 9:'9', 10:'10', 11:'11', 12:'12'}

# Final columns, except Grade, Total
datacols = [ 
  'School Name',
  # 'Street Address', # Missing in some
  # Some are "District Name", some are "City"
  # Will use to hold both and search for substrings: "Bellevue" 
  "Region",
  # 'State',
  # 'Zipcode', Missing in 2018
  # 'County'
  'Year', # Added 
]
allFinalCols = datacols + ["Grade", "Total"]
# Final grade categories
grades= ["P", "K"] + [str(g) for g in range(1, 13)]

In [ ]:
def checkNulls(df):
    schools = set()
    for col in ['Grade', 'Year', 'Total', 
                'School Name',
                'Region', 
#                 'Street Address'
               ]:
        mask = (df[col].isna() | df[col].isnull() )
        if mask.sum():
            print(f"{col} has {len(mask)} nulls")
            schools.intersection(df.loc[mask, "School Name"].unique())
    for sname in schools:
        display(df[df["School Name"]==sname])
#                 display(df.loc[mask])

In [ ]:
# Normalize 2018 --- Not used. Format is quite a lot difference. Suspicious of the changes
# d2018_ = d2018.rename(columns={"District Name": "Region"}) # "City" is missing in this dataset. We only have "District Name"
# d2018_['Grade'] = d2018['Grade'].replace(gradeMap)
# d2018_['Year'] = 2018
# d2018_ = d2018_[d2018_['School Name'].notnull()]
# print(d2018_.shape, list(d2018_[allFinalCols].columns))

In [ ]:
# Normalize 2019
df = d2019.rename(columns=gradeMap)
df = df.rename(columns={"Total":"_Total", "Name of School": "School Name", "City": "Region"})
# print(sorted(df.columns))
df = df.melt(id_vars=set(df.columns)-set(grades), value_vars=grades, var_name="Grade", value_name="Total")
df['Year'] = 2019
df = df[df['School Name'].notnull()]
print(df.shape, list(df[allFinalCols].columns))
checkNulls(df)
d2019_ = df

In [ ]:
# Normalize 2020
df = d2020.rename(columns=gradeMap).rename(columns={"City": "Region"})
print(df.shape, list(df.columns))
df[grades]  # The grade columns should all exist
df = df.melt(id_vars=set(df.columns)-set(grades), value_vars=grades, var_name="Grade", value_name="Total")
df['Grade'] = df['Grade'].replace(gradeMap)
df['Year'] = 2020
df = df[df['School Name'].notnull()]
checkNulls(df)
# display(df[df["Region"].isnull()])
# display(d2020["City"].isnull().sum())
print(f'null cities: {d2020["City"].isnull().sum()}')
# print(f'Schools with null cities:')
# for school in d2020.loc[d2020["City"].isnull(),"School Name"].unique():
#     display(d2020[d2020["School Name"]==school])
# print(f'Schools with null totals:')
# for school in df.loc[df["Total"].isnull(),"School Name"].unique():
#     display(d2020[d2020["School Name"]==school])

# display(df[df["Total"].isnull()])
# display(df[df["Region"].isnull() & (df["School Name"]=="Carden Country School")])
# display(df[df["School Name"]=="Carden Country School"])
d2020_ = df

In [ ]:
# Normalize 2021
def _normalize(df):
    df = df.rename(columns=gradeMap).rename(columns={"Name of School": "School Name", 'ZIP': "Zipcode", "City": "Region"})
    df[grades]  # The grade columns should all exist
    df = df.melt(id_vars=set(df.columns)-set(grades), value_vars=grades, var_name="Grade", value_name="Total")
    df['Grade'] = df['Grade'].replace(gradeMap)
    df['Year'] = 2021
    df.loc[df["Total"].isna(), "Total"] = 0
    df["Total"] = df["Total"].astype(int)

    df = df[df['School Name'].notnull()]
    checkNulls(df)
    print(df.shape, list(df[allFinalCols].columns))
    return df
print(d2021.columns)
d2021_ = _normalize(d2021)

In [ ]:
# Normalize 2022
def _normalize(df):
    df = df.rename(columns=gradeMap).rename(columns={'ZIP': "Zipcode", "City": "Region", "Address": "Street Address"})
    print(df.columns)
    df[grades]  # The grade columns should all exist
    df = df.melt(id_vars=set(df.columns)-set(grades), value_vars=grades, var_name="Grade", value_name="Total")
    df['Grade'] = df['Grade'].replace(gradeMap)
    df['Year'] = 2022
    df.loc[df["Total"].isna(), "Total"] = 0
    df["Total"] = df["Total"].astype(int)
    df = df[df['School Name'].notnull()]

    checkNulls(df)
#     print(df.shape, list(df[allFinalCols].columns))
    print(df.shape, list(df[allFinalCols].columns))
    return df

# Index(['School Name', 'Street Address', 'Region', 'State', 'Zipcode', 'P',
#        'PK Total.1', 'K', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10',
#        '11', '12', 'K12 Total Reported'],
#       dtype='object')
d2022_ = _normalize(d2022)

# Index(['School Name', 'Street Address', 'Region', 'State', 'Zipcode', 'County',
#        'District', 'PreK TOTAL', 'KG TOTAL', 'G1 TOTAL', 'G2 TOTAL',
#        'G3 TOTAL', 'G4 TOTAL', 'G5 TOTAL', 'G6 TOTAL', 'G7 TOTAL', 'G8 TOTAL',
#        'G9 TOTAL', 'GI0 TOTAL', 'G11 TOTAL', 'G12 TOTAL'],
#       dtype='object')
d2023_ = _normalize(d2023)

In [ ]:
df = d2023_
for c in df.columns:
    df = df[df[c].isna()]
    if len(df)==0: continue
    display(df)

In [ ]:
def normalizeAddresses(df):
    df = df.copy()
    def simplifyAddr(a):
        try:
            a = a.upper().replace(".", "").replace("AVENUE", "AVE")
            a = " ".join(a.split()[:3])
            return a
        except:
            print("Faied address", a)
            raise
    df["Street Address"] = df["Street Address"].apply(simplifyAddr)
    return df

def fixDuplicateAddress(df):
    df_ = [["Region", "School Name", "Street Address"]].sort_values(["Region", "School Name", "Street Address"])\
        .drop_duplicates() \
        .reset_index()
    for idx, region, school, addr in df_[df_[["Region", "School Name"]].duplicated(keep=False)].itertuples():
        mask = (df["Region"]==region) & (df["School Name"]==school)
        newschool = f"{school} {addr}"
        print(f"Renaming {repr(school)}, {repr(addr)} -> {repr(newschool)}")
        df.loc[mask, "School Name"] = newschool

# 2018 seems to have some very different reporting. Not sure it is consistent. Dropped here
dAll = pd.concat([d2019_, d2020_, d2021_, d2022_]).reindex()
# dAll = pd.concat([d2022_, ]).reindex()
# Place the important data in the first columns. 
dAll = dAll[allFinalCols + list(set(dAll.columns)-set(allFinalCols))].sort_values(["Year", "Grade", "School Name"])
dAll = dAll[~(dAll["School Name"].isnull())] # Zap rows without school names
dAll = dAll[~(dAll["School Name"].astype(str)=='NaN')] # Zap rows without school names
dAll = dAll[~(dAll["School Name"].isna())] # Zap rows without school names
dAll = dAll[dAll["School Name"]!="State Total"] # Zap total rows
dAll.loc[dAll["Total"].isna(), "Total"] = 0
dAll = dAll.astype({"Total": int})
dAll = dAll.reset_index()

# display(dAll[dAll["Street Address"].isna()])
# dAll = normalizeAddresses(dAll)
# dAll = fixDuplicateAddress(dAll)

# There are a number of school in the data without a district or city, or a district of 0 for some reason.
# We'll just ignore them from bellevue processing since none are in Bellevie
# dAll = dAll[dAll["Region"].astype(str)!='0'] # 0 is not a region
print(f"Years reporting {dAll['Year'].unique()}")

print("Schools reporting their city/district as 0")
display(dAll[dAll["Region"].isnull()]["School Name"].unique())

print("\nTotal WA private enrollment by year")
display(dAll.groupby("Year")["Total"].sum())

In [ ]:
print(f'Regions considered:')

dAllRegionOfInterest = dAll[["Grade", "School Name", "Region", "Total", "Year"]]
dAllRegionOfInterest = dAllRegionOfInterest[~(dAllRegionOfInterest["Region"].isnull())]
dAllRegionOfInterest = dAllRegionOfInterest[dAllRegionOfInterest["Region"].astype(str).str.contains(regionSubstring)]
dAllRegionOfInterest["School Name"] = dAllRegionOfInterest["School Name"].str.strip().replace(
     {"SARODGINI CHILDREN'S ACADEMY": "Sarodgini Children's Academy",
     "Dartmoor School - Bellevue": "Dartmoor School",
     "Cedar Park Christian - Bellevue": "Cedar Park Christian School",
      "Living Montessori": "Living Montessori Education Community"
     })

zeroReportedAllYear = dAllRegionOfInterest.groupby("School Name")["Total"].sum()
zeroReportedAllYear = zeroReportedAllYear[zeroReportedAllYear==0]
print(f'\nSchool reporting 0 enrollment (all year): {len(zeroReportedAllYear)}')
display(zeroReportedAllYear.to_frame())
#display(dAllRegionOfInterest[dAllRegionOfInterest["School Name"].isin(zeroReportedAnyYear.index)])

# Removes schools not reporting any data at all
# print(len(dAllRegionOfInterest["School Name"].unique()), len(dAllRegionOfInterest), dAllRegionOfInterest["Total"].sum())
#dAllRegionOfInterest = dAllRegionOfInterest[~(dAllRegionOfInterest["School Name"].isin(zeroReportedAnyYear.index))]
# print(len(dAllRegionOfInterest["School Name"].unique()), len(dAllRegionOfInterest), dAllRegionOfInterest["Total"].sum())

for school in sorted(dAllRegionOfInterest["School Name"].unique()):
    print(school, dAllRegionOfInterest.loc[dAllRegionOfInterest["School Name"]==school, "Total"].sum())
print("====")
    
zeroReportedAllYear = dAllRegionOfInterest.groupby("School Name")["Total"].sum()
zeroReportedAllYear = zeroReportedAllYear[zeroReportedAllYear==0]
print(f'\nSchool reporting 0 enrollment (all year): {zeroReportedAllYear.size}')


regions = dAllRegionOfInterest["Region"].unique()
display(regions)


In [ ]:

print(f"\nTotal {', '.join(regions)} private enrollment by year")
display(dAllRegionOfInterest.groupby("Year")["Total"].sum().to_frame())

print(f'\nSchools reporting: {dAllRegionOfInterest["School Name"].unique().size}.')
schools = sorted(dAllRegionOfInterest["School Name"].unique())
for school in schools:
    print(school)
    
# display(schools)


# print(df["Year"].dtype)
for year in range(2019, 2022+1):
    df = dAllRegionOfInterest[dAllRegionOfInterest["Year"]==year]

# display(dAllRegionOfInterest.groupby("Year")["Total"].sum())
# for year, df in dAllRegionOfInterest.groupby("Year"):
#     print(f"{year} ------------")
#     display(df.groupby("School Name")["Total"].sum())

# for school, df in dAllRegionOfInterest.groupby("School Name"):
#     for year, df in df.groupby("Year"):
#         display(df["Total"].sum().to_frame())
# for (school,year), df in dAllRegionOfInterest.groupby(["School Name", "Year"]):
#     #for year, df in df.groupby("Year"):
#         display(df["Total"].sum().to_frame())
print("-----------")

for year in range(2019, 2022+1):
    reportingZero = []
    df = dAllRegionOfInterest[dAllRegionOfInterest["Year"]==year]
    for school in schools:
        total = df[df["School Name"]==school]["Total"].sum()
        print(f"{year}: Total:{total} {school}")
        if total==0:
            reportingZero.append(school)
    print(f"Total: {df['Total'].sum()}")
    print(f'\nSchool reporting 0 enrollment ({year}): {len(reportingZero)}')
    display(reportingZero)


# The ArtSci Company  No data
# Ventures Academy    No data

# 'BASIS Independent Bellevue', opened 2022/23
# "Bellevue Children's Academy", opened 1999. Reopened onsite 2021
# 'Eastside Community School',  # Colocated with international friends school? Moved to Issaquah
# International Montessori Academy - Eastgate   #Closed 2019,2020?
# International Friends school  # Opened 2018. Open during covid, but not reporting
# 'Seattle VocTech School',   ~10 students
# 'Tarteel School'  Closed 2020/21?   Only data from 2022/23? (180 students)

In [ ]:
# Show enrollment changes from baseline year
import matplotlib.pyplot as plt
import matplotlib.colors as _colors  
import itertools, random
import matplotlib.ticker as mticker
import math

baselineYear = 2019
allGrades = ["P","K"]+[str(s) for s in range(1,13)]

colors = list(_colors.TABLEAU_COLORS.values())
random.Random(0).shuffle(colors)
styles = itertools.cycle(itertools.product(['dashdot', 'solid', 'dashed', 'dashdot', 'dotted'], colors))

_school2style = {}
def school2style(school):
    s = _school2style.get(school)
    if s is None:
        s = next(styles)
#         print(s)
        _school2style[school] = s
    return s
[school2style(s) for s in sorted(dAllRegionOfInterest["School Name"].unique())]

#def plot(genFunc, baselineYear, selectGrades, title, xlabel, ylabel):
# 1:
xlabel = "Year"
ylabel = "Enrollment"
df = dAllRegionOfInterest.copy()
for school, sgrp in df.groupby("School Name"):
    f = plt.figure()
    f.set_figwidth(10)
    f.set_figheight(10)

    print(school)
    
#     df = dAllRegionOfInterest #["Grade"]
#     df = df[df["School Name"]=="America's Child Montessori"]
    # display(df)   
    # df = df.sort_values(by="Grade", key=lambda col:col.map(grade2Order)).set_index("Year")
    # for grade, grp in df.groupby("Grade"):
    display(sgrp.groupby("Year")['Total'].sum().to_frame())
    sgrp = sgrp.set_index("Year")
    sgrp = sgrp[sgrp["Total"]!=0]
    print(f"Total: {sgrp['Total'].sum()}")
    display(sgrp)
    for grade in allGrades:
        df_ = sgrp[sgrp["Grade"]==grade]["Total"]
        if len(df_)==0: continue
        if df_.sum()==0: continue
        y = df_.values
        x = df_.index
#         print(grade)
#         display(df_)
        linestyle, color = school2style(school)
        plt.plot(x, y, linestyle=linestyle, 
                 #color=color,
                 marker='o',
                label=grade)

    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(school)
    plt.gca().xaxis.set_major_locator(mticker.MultipleLocator(1))
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.show()
#     break
        
    
    
# for name, select in [
#     ("P-3", ["P", "K", "1", "2", "3"]),
#     ("K-5", ["K", "1", "2", "3"," 4", "5"]),
#     ("P-5", ["P", "K", "1", "2", "3", "4", "5"]),
#     ("K-3", ["K", "1", "2", "3"]),
#     ("K-12", ["K"]+[str(s) for s in range(1,13)]),
#     ("P-12", allGrades),
#     ("All", None),  # Sanity check, should match P-12
#   ] + [(g, [g]) for g in grades]:
    
#   if select:
#     bvDiff = dAllRegionOfInterest[dAllRegionOfInterest["Grade"].isin(select)].copy()
#   else:
#     bvDiff = bvDiff.copy()
#   bvDiff = bvDiff[bvDiff['School Name']=="America's Child Montessori"]
#   display(bvDiff)

#   bvDiffTotals = bvDiff.groupby("Year")["Total"].sum().to_frame()

#   display(bvDiffTotals)
#   totalName = f"'Grade {name}' private enrollment, Δ since {baselineYear}"
#   bvDiffTotals[totalName] = bvDiffTotals["Total"] - bvDiffTotals["Total"][baselineYear]
#   bvDiffTotals[f"%change"] = 100*(bvDiffTotals[totalName] / bvDiffTotals["Total"][baselineYear])
  
#   g = bvDiff.groupby(["School Name", "Year"])["Total"].sum().to_frame().reset_index()
      
#   bvDiffTotals[f"# schools reporting"] = g.groupby(["Year"]).count()["Total"]
#   bvDiffTotals[f"# reporting 0"] = g[g["Total"]==0].groupby(["Year"]).count()["Total"]
#   bvDiffTotals[f"# reporting not 0"] = g[g["Total"]!=0].groupby(["Year"]).count()["Total"]

#   display(bvDiffTotals)

In [ ]:
df = sgrp.set_index("Year")
for grade in allGrades:
    df_ = sgrp[sgrp["Grade"]==grade]["Total"]
    if len(df_)==0: continue
    if df_.sum()==0: continue

In [ ]:

grade2Order = {g:o for o,g in enumerate(allGrades)}.get

allGrades = ["P", "K"]+[str(s) for s in range(12)]
df = dAllRegionOfInterest #["Grade"]
df = df[df["School Name"]=="America's Child Montessori"]
# display(df)   
# df = df.sort_values(by="Grade", key=lambda col:col.map(grade2Order)).set_index("Year")
# for grade, grp in df.groupby("Grade"):
df = df.set_index("Year")
for grade in allGrades:
    df_ = df[df["Grade"]==grade]
    print(grade)
    if len(df_)==0: continue
    if df_["Total"].sum()==0: continue
    display(df_["Total"])
#     df = df.sort_values(by="Grade", key=lambda col:col.map(grade2Order))
df
# dAllRegionOfInterest["Year"].unique()

In [ ]:
grade2Order("K")

In [ ]:
grp

In [ ]:
# Show enrollment changes from baseline year
baselineYear = 2019
allGrades = ["P","K"]+[str(s) for s in range(1,13)]
for name, select in [
    ("P-3", ["P", "K", "1", "2", "3"]),
    ("K-5", ["K", "1", "2", "3"," 4", "5"]),
    ("P-5", ["P", "K", "1", "2", "3", "4", "5"]),
    ("K-3", ["K", "1", "2", "3"]),
    ("K-12", ["K"]+[str(s) for s in range(1,13)]),
    ("P-12", allGrades),
    ("All", None),  # Sanity check, should match P-12
  ] + [(g, [g]) for g in grades]:
    
  if select:
    bvDiff = dAllRegionOfInterest[dAllRegionOfInterest["Grade"].isin(select)].copy()
  else:
    bvDiff = bvDiff.copy()
  bvDiff = bvDiff[bvDiff['School Name']=="America's Child Montessori"]
  display(bvDiff)

  bvDiffTotals = bvDiff.groupby("Year")["Total"].sum().to_frame()

  display(bvDiffTotals)
  totalName = f"'Grade {name}' private enrollment, Δ since {baselineYear}"
  bvDiffTotals[totalName] = bvDiffTotals["Total"] - bvDiffTotals["Total"][baselineYear]
  bvDiffTotals[f"%change"] = 100*(bvDiffTotals[totalName] / bvDiffTotals["Total"][baselineYear])
  
  g = bvDiff.groupby(["School Name", "Year"])["Total"].sum().to_frame().reset_index()
      
  bvDiffTotals[f"# schools reporting"] = g.groupby(["Year"]).count()["Total"]
  bvDiffTotals[f"# reporting 0"] = g[g["Total"]==0].groupby(["Year"]).count()["Total"]
  bvDiffTotals[f"# reporting not 0"] = g[g["Total"]!=0].groupby(["Year"]).count()["Total"]

  display(bvDiffTotals)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as _colors  
import itertools, random
import matplotlib.ticker as mticker
import math

print("Full data")
def gen():
    df = dAllRegionOfInterest[["Year", "School Name", "Grade", "Total"]].sort_values(["Year", "School Name", "Grade"])
    for (school, year), dfbyschool in df.groupby(["School Name", "Year"]):
        if school in ['Bellevue Montessori School', 'International Montessori Academy', 'Sacred Heart School']: continue
        dfbyschool["X"] = f"{year} {school}"
        display(dfbyschool)
        d = dfbyschool.pivot(index="X", columns="Grade",values="Total")
        display(d)



with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
#     gen()
    
#     df = pd.concat(gen())
#     display(df)

    for (school, year), dfbyschool in df.groupby(["School Name", "Year"]):
#         print(school)
#         dfbyschool["X"] = f"{year} {school}"
#         display(dfbyschool.pivot(index="X", columns="Grade",values="Total"))
#         # display(df[["Grade", "Total"]].set_index("Grade").pivot(
#         #          # columns="Grade",
#         #          values="Total"))
#         break

        f = plt.figure()
        f.set_figwidth(5)
        f.set_figheight(5)
        for grade, dfbygrade in dfbyschool.groupby(["Grade"]):
           x, y = dfbygrade["Year"].values, dfbygrade["Total"].values
           if y.sum()==0:
              print(f"No data for grade {grade}")
              continue
           plt.plot(x, y, #linestyle=linestyle, color=color, 
                    label=grade)
        print(x)
        print(y)
        plt.show()            
        break
        plt.xlabel('Year')
        plt.ylabel('Enrollment')
        plt.title(school)                
        plt.gca().xaxis.set_major_locator(mticker.MultipleLocator(1))
        plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
        plt.show()            
#         # print(dfbyyear)

# .pivot(#index=, 
#                                      columns="Grade",
#                                      values="Total"))

  # display(dAllRegionOfInterest[["School Name", "Grade", "Year", "Total"]]))

In [ ]:
# print(df_)

In [ ]:
# df_.loc[mask.values, "School Name"]
mask.shape, df_.shape

In [ ]:
mask.sum()